In [1]:
class CFG:
    exps = ['026', '022']
    n_fold = 4
    debug=False
    seed = 42

# ====================================================
# Colab settings
# ====================================================
class ColabConfig:
    dataset_name = 'PPPM-stacking'
    dataset_version = 'exp' + '-'.join(CFG.exps)
    dataset_new = True  # 新しいデータセットか
    dataset_dir = None  # Kaggle Dataset にアップロードするディレクトリ
    dataset_note = '""'  # 前の版からの変更点
    in_colab = False  # colab上にデータダウンロード

In [2]:
!nvcc --version
!python -c 'import torch; print(torch.__version__) '
!python --version
print('')
!nvidia-smi
print('')
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
1.11.0+cu113
Python 3.7.13

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.


Your runtime has 13.6 gigabytes of available RAM



In [3]:
%%time
import sys
COLAB = "google.colab" in sys.modules

if COLAB:
    import os
    print('This environment is Google Colab')

    # mount drive
    from google.colab import drive
    if not os.path.isdir('/content/drive'):
        drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/kaggle/PPPM/exps')

    # kaggle api token and update kaggle api
    from google.colab import files
    if not os.path.isfile('~/.kaggle/kaggle.json'):
        # files.upload()
        !mkdir -p ~/.kaggle
        !cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json
        !pip install --upgrade --force-reinstall --no-deps -q kaggle

    if ColabConfig.in_colab:
        # make directory in colab
        !mkdir -p /content/input
        !mkdir -p /content/working

        # download dataset in colab
        import zipfile, glob
        os.chdir('/content/input')
        # !kaggle competitions download -qc birdclef-2022 -p birdclef-2022
        
        # !mkdir birdclef-2022
        # !cp /content/drive/MyDrive/kaggle/BirdCLEF2022/input/birdclef-2022/* ./birdclef-2022/

        for p in glob.glob('**/*.zip', recursive=True):
            print(p)
            d, f = os.path.split(p)
            # if f in ['']:
            #     continue
            with zipfile.ZipFile(p, 'r') as zipf:
                print('unzip: ', zipf)
                zipf.extractall(d)
                print('remove: ', f)
                os.remove(p)
        
        os.chdir('/content/working')

This environment is Google Colab
Mounted at /content/drive
     |████████████████████████████████| 58 kB 6.4 MB/s 
CPU times: user 1.43 s, sys: 267 ms, total: 1.7 s
Wall time: 20.9 s


In [4]:
import os
import random
import pickle
from pathlib import Path
from psutil import cpu_count
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 10000)
pd.set_option('display.max_colwidth', None)
from tqdm.auto import tqdm
tqdm.pandas()
os.system('python -m pip install -U lightgbm')
import lightgbm as lgb
print(f"lightgbm.__version__: {lgb.__version__}")
os.system('python -m pip install optuna')
import optuna.integration.lightgbm as lgb_optuna
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
import torch
print(f"torch.__version__: {torch.__version__}")

os.system('python -m pip install sentencepiece')
os.system('pip uninstall -y transformers')
os.system('pip uninstall -y tokenizers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels transformers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels tokenizers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

lightgbm.__version__: 3.3.2
torch.__version__: 1.11.0+cu113
tokenizers.__version__: 0.12.1
transformers.__version__: 4.18.0
env: TOKENIZERS_PARALLELISM=true


In [5]:
input_dir = Path('../input/us-patent-phrase-to-phrase-matching')
output_dir = Path(f"/content/drive/MyDrive/kaggle/PPPM/output/stacking{'_'.join(CFG.exps)}")
output_dir.mkdir(exist_ok=True)

ColabConfig.dataset_dir = str(output_dir)

In [6]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = sp.stats.pearsonr(y_true, y_pred)[0]
    return score


def get_logger(filename=output_dir / 'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

# LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [7]:
# 特徴量エンジニアリング
def tok_len(sentence, tokenizer):
    return len(tokenizer.tokenize(sentence))

def tokenizer_feature_engineering(df, tokenizer):
    def _tok_len(sentence):
        return tok_len(sentence, tokenizer)
    
    basecols = df.columns

    df['anchor_tok_len'] = df['anchor'].map(_tok_len)
    df['target_tok_len'] = df['target'].map(_tok_len)
    df['context_tok_len'] = df['context_text'].map(_tok_len)
    df['input_len'] = df['anchor_tok_len'] + df['target_tok_len'] + df['context_tok_len'] + 4

    df['len_anc_tgt_diff'] = df['anchor_tok_len'] - df['target_tok_len']
    df['len_anc_tgt_div'] = df['anchor_tok_len'] / df['target_tok_len']

    df['len_anc_cnt_diff'] = df['anchor_tok_len'] - df['context_tok_len']
    df['len_anc_cnt_div'] = df['anchor_tok_len'] / df['context_tok_len']

    df['len_tgt_cnt_diff'] = df['target_tok_len'] - df['context_tok_len']
    df['len_tgt_cnt_div'] = df['target_tok_len'] / df['context_tok_len']

    # 作成した特徴量とidカラムだけ返す
    usecols = [col for col in df.columns if col not in basecols]
    usecols.append('id')
    return df[usecols]

In [8]:
def make_ensemble_datasets(cfg):
    cv_scores = []
    train = pd.read_csv(input_dir / 'train.csv')

    for exp in cfg.exps:
        oof_dir = Path(f'/content/drive/MyDrive/kaggle/PPPM/output/{exp}')
        oof_df = pd.read_pickle(oof_dir / 'oof_df.pkl')
        tokenizer = AutoTokenizer.from_pretrained(oof_dir / 'tokenizer')
        
        # 特徴量と予測スコアは'pred_000'のようなカラム名にする
        feature_df = tokenizer_feature_engineering(oof_df, tokenizer)
        rename_dict = {col: col+'_'+exp for col in feature_df.columns if col != 'id'}
        feature_df = feature_df.rename(columns=rename_dict)
        
        train = train.merge(feature_df, on='id', how='left')
        train = train.merge(oof_df[['id', 'pred']].rename(columns={'pred': f'pred_{exp}'}), 
                            on='id', how='left')

        cv_scores.append(get_score(train['score'], train[f'pred_{exp}']))
    
    print(f'max score: {max(cv_scores)}')
    print(f'avg score: {sum(cv_scores) / len(cv_scores)}')
    return train

In [9]:
train = make_ensemble_datasets(CFG)
display(train.head())

max score: 0.832151085616115
avg score: 0.831917956699366


,id,anchor,target,context,score,anchor_tok_len_026,target_tok_len_026,context_tok_len_026,input_len_026,len_anc_tgt_diff_026,len_anc_tgt_div_026,len_anc_cnt_diff_026,len_anc_cnt_div_026,len_tgt_cnt_diff_026,len_tgt_cnt_div_026,pred_026,anchor_tok_len_022,target_tok_len_022,context_tok_len_022,input_len_022,len_anc_tgt_diff_022,len_anc_tgt_div_022,len_anc_cnt_diff_022,len_anc_cnt_div_022,len_tgt_cnt_diff_022,len_tgt_cnt_div_022,pred_022
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,1,3,20,28,-2,0.333333,-19,0.05,-17,0.15,0.440476,1,3,20,28,-2,0.333333,-19,0.05,-17,0.15,0.126895
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,1,5,20,30,-4,0.200000,-19,0.05,-15,0.25,0.641294,1,5,20,30,-4,0.200000,-19,0.05,-15,0.25,0.632196
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0.247038,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0.237127
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0.494176,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0.489984
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0.011766,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0.013014


In [10]:
# foldに分ける
# credits to: https://www.kaggle.com/code/abhishek/creating-folds-properly-hopefully-p

!pip install -q iterative-stratification
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

dfx = pd.get_dummies(train, columns=["score"]).groupby(["anchor"], as_index=False).sum()
cols = [c for c in dfx.columns if c.startswith("score_") or c == "anchor"]
dfx = dfx[cols]

mskf = MultilabelStratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=42)
labels = [c for c in dfx.columns if c != "anchor"]
dfx_labels = dfx[labels]
dfx["fold"] = -1

for fold, (trn_, val_) in enumerate(mskf.split(dfx, dfx_labels)):
    print(len(trn_), len(val_))
    dfx.loc[val_, "fold"] = fold

train = train.merge(dfx[["anchor", "fold"]], on="anchor", how="left")
print(train.fold.value_counts())

550 183
549 184
550 183
550 183
3    9622
0    9379
1    8860
2    8612
Name: fold, dtype: int64


In [11]:
# カテゴリ変数の変換
train['section'] = train['context'].map(lambda s: s[0])
cat_cols = ['section']
for c in cat_cols:
    le = LabelEncoder()
    le.fit(train[c])
    train[f'{c}_encoded'] = le.transform(train[c])
    with open(output_dir / f'{c}_encoder.pkl', 'wb') as f:
        pickle.dump(le, f)

In [12]:
print(train.columns)

Index(['id', 'anchor', 'target', 'context', 'score', 'anchor_tok_len_026', 'target_tok_len_026', 'context_tok_len_026', 'input_len_026', 'len_anc_tgt_diff_026', 'len_anc_tgt_div_026', 'len_anc_cnt_diff_026', 'len_anc_cnt_div_026', 'len_tgt_cnt_diff_026', 'len_tgt_cnt_div_026', 'pred_026', 'anchor_tok_len_022', 'target_tok_len_022', 'context_tok_len_022', 'input_len_022', 'len_anc_tgt_diff_022', 'len_anc_tgt_div_022', 'len_anc_cnt_diff_022', 'len_anc_cnt_div_022', 'len_tgt_cnt_diff_022', 'len_tgt_cnt_div_022', 'pred_022', 'fold', 'section', 'section_encoded'], dtype='object')


In [13]:
excluded_columns = ['id', 'anchor', 'target', 'context', 'score', 'fold', 'section']
cols = [col for col in train.columns if col not in excluded_columns]
with open(output_dir / 'cols.pkl', 'wb') as f:
    pickle.dump(cols, f)
print(cols)

['anchor_tok_len_026', 'target_tok_len_026', 'context_tok_len_026', 'input_len_026', 'len_anc_tgt_diff_026', 'len_anc_tgt_div_026', 'len_anc_cnt_diff_026', 'len_anc_cnt_div_026', 'len_tgt_cnt_diff_026', 'len_tgt_cnt_div_026', 'pred_026', 'anchor_tok_len_022', 'target_tok_len_022', 'context_tok_len_022', 'input_len_022', 'len_anc_tgt_diff_022', 'len_anc_tgt_div_022', 'len_anc_cnt_diff_022', 'len_anc_cnt_div_022', 'len_tgt_cnt_diff_022', 'len_tgt_cnt_div_022', 'pred_022', 'section_encoded']


In [14]:
params = {
    'objective': 'regression',
    'boosting': 'gbdt',
    'num_iterations': 1000,
    'learning_rate': 0.01,
    # 'num_leaves': 20,  # 31
    'num_threads': cpu_count(logical=False),
    'seed': CFG.seed,

    # 'max_depth': 3,  # -1
    # 'min_data_in_leaf': 100,  # 20
    'verbosity': -1,  # 1

    'metric': 'l2'
}

In [15]:
def tuning_params(df, params):
    anchors = df['anchor'].unique()
    rng = np.random.default_rng(seed=71)
    rng.shuffle(anchors)
    val_prop = 0.25  # バリデーションデータの割合。4foldから0.25
    val_size = int(len(anchors)*val_prop)
    val_anchors = anchors[:val_size]
    is_val = np.isin(df['anchor'], val_anchors)
    idx = np.arange(len(df))
    val_idx = idx[is_val]
    tr_idx = idx[~is_val]
    print(f'train: {len(tr_idx)}, val: {len(val_idx)}')

    tr_df = df.iloc[tr_idx]
    val_df = df.iloc[val_idx]

    cat_cols_ = [f'{c}_encoded' for c in cat_cols]
    tr_ds = lgb.Dataset(tr_df[cols], tr_df['score'], categorical_feature=cat_cols_)
    val_ds = lgb.Dataset(val_df[cols], val_df['score'], categorical_feature=cat_cols_, reference=tr_ds)
    
    model = lgb_optuna.train(
        params,
        tr_ds, 
        valid_sets=[tr_ds, val_ds],
        callbacks=[lgb.early_stopping(stopping_rounds=100),
                    lgb.log_evaluation(period=100)],
        verbosity=-1,
    )

    best_params = model.params

    return best_params

In [ ]:
best_params = tuning_params(train, params)
display(best_params)

train: 27261, val: 9212


[I 2022-06-19 15:02:36,780] A new study created in memory with name: no-name-dc39e5e2-55fd-483f-9899-3f29b9e9a9a5
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

[1]	valid_0's l2: 0.066289	valid_1's l2: 0.0643226
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l2: 0.065357	valid_1's l2: 0.0634219
[3]	valid_0's l2: 0.0644434	valid_1's l2: 0.0625391
[4]	valid_0's l2: 0.063548	valid_1's l2: 0.0616742
[5]	valid_0's l2: 0.0626705	valid_1's l2: 0.0608267
[6]	valid_0's l2: 0.0618106	valid_1's l2: 0.059996
[7]	valid_0's l2: 0.0609675	valid_1's l2: 0.0591825
[8]	valid_0's l2: 0.0601412	valid_1's l2: 0.0583851
[9]	valid_0's l2: 0.0593415	valid_1's l2: 0.0576107
[10]	valid_0's l2: 0.0585475	valid_1's l2: 0.0568445
[11]	valid_0's l2: 0.0577793	valid_1's l2: 0.0561018
[12]	valid_0's l2: 0.057016	valid_1's l2: 0.0553659
[13]	valid_0's l2: 0.056268	valid_1's l2: 0.0546451
[14]	valid_0's l2: 0.055535	valid_1's l2: 0.0539361
[15]	valid_0's l2: 0.0548161	valid_1's l2: 0.0532437
[16]	valid_0's l2: 0.054112	valid_1's l2: 0.0525632
[17]	valid_0's l2: 0.0534214	valid_1's l2: 0.0518988
[18]	valid_0's l2: 0.0527449	valid_1's l2: 0.0512454
[

feature_fraction, val_score: 0.019582:  14%|#4        | 1/7 [00:03<00:21,  3.64s/it]

[475]	valid_0's l2: 0.0180298	valid_1's l2: 0.0196014
[476]	valid_0's l2: 0.0180281	valid_1's l2: 0.0196009
[477]	valid_0's l2: 0.0180258	valid_1's l2: 0.0196015
[478]	valid_0's l2: 0.0180234	valid_1's l2: 0.019601
[479]	valid_0's l2: 0.0180211	valid_1's l2: 0.0196012
[480]	valid_0's l2: 0.0180191	valid_1's l2: 0.0196011
[481]	valid_0's l2: 0.0180169	valid_1's l2: 0.0196014
[482]	valid_0's l2: 0.0180153	valid_1's l2: 0.0196012
Early stopping, best iteration is:
[382]	valid_0's l2: 0.0182668	valid_1's l2: 0.0195818
[1]	valid_0's l2: 0.0663026	valid_1's l2: 0.0643319
[2]	valid_0's l2: 0.0653705	valid_1's l2: 0.0634303
[3]	valid_0's l2: 0.06447	valid_1's l2: 0.0625572
[4]	valid_0's l2: 0.0635875	valid_1's l2: 0.0617013
[5]	valid_0's l2: 0.062721	valid_1's l2: 0.0608627
[6]	valid_0's l2: 0.0626246	valid_1's l2: 0.0607904
[7]	valid_0's l2: 0.0617768	valid_1's l2: 0.0599702
[8]	valid_0's l2: 0.0609463	valid_1's l2: 0.0591647
[9]	valid_0's l2: 0.0608567	valid_1's l2: 0.0590999
[10]	valid_0's 

...

[491]	valid_0's l2: 0.0184666	valid_1's l2: 0.0195469
[492]	valid_0's l2: 0.0184648	valid_1's l2: 0.0195467
[493]	valid_0's l2: 0.0184634	valid_1's l2: 0.0195461
Early stopping, best iteration is:
[393]	valid_0's l2: 0.0186548	valid_1's l2: 0.0195286
[1]	valid_0's l2: 0.066299	valid_1's l2: 0.0643293
[2]	valid_0's l2: 0.0653765	valid_1's l2: 0.0634348
[3]	valid_0's l2: 0.0644722	valid_1's l2: 0.0625585
[4]	valid_0's l2: 0.0635862	valid_1's l2: 0.0616994
[5]	valid_0's l2: 0.0627144	valid_1's l2: 0.0608558
[6]	valid_0's l2: 0.0618599	valid_1's l2: 0.060029
[7]	valid_0's l2: 0.0610222	valid_1's l2: 0.059218
[8]	valid_0's l2: 0.0602014	valid_1's l2: 0.058423
[9]	valid_0's l2: 0.0593966	valid_1's l2: 0.0576438
[10]	valid_0's l2: 0.0586078	valid_1's l2: 0.0568803
[11]	valid_0's l2: 0.0578349	valid_1's l2: 0.0561301
[12]	valid_0's l2: 0.0570768	valid_1's l2: 0.0553974
[13]	valid_0's l2: 0.0563351	valid_1's l2: 0.0546819
[14]	valid_0's l2: 0.0556081	valid_1's l2: 0.0539808
[15]	valid_0's l2: 0

min_data_in_leaf, val_score: 0.019529:  80%|########  | 4/5 [00:15<00:03,  3.93s/it]

[483]	valid_0's l2: 0.018563	valid_1's l2: 0.0195555
[484]	valid_0's l2: 0.0185619	valid_1's l2: 0.0195552
[485]	valid_0's l2: 0.0185607	valid_1's l2: 0.0195558
[486]	valid_0's l2: 0.0185595	valid_1's l2: 0.0195564
[487]	valid_0's l2: 0.0185584	valid_1's l2: 0.019557
[488]	valid_0's l2: 0.0185573	valid_1's l2: 0.0195577
[489]	valid_0's l2: 0.0185553	valid_1's l2: 0.0195578
[490]	valid_0's l2: 0.0185533	valid_1's l2: 0.0195579
[491]	valid_0's l2: 0.0185513	valid_1's l2: 0.0195583
[492]	valid_0's l2: 0.0185496	valid_1's l2: 0.0195587
[493]	valid_0's l2: 0.0185479	valid_1's l2: 0.0195589
Early stopping, best iteration is:
[393]	valid_0's l2: 0.0186548	valid_1's l2: 0.0195286
[1]	valid_0's l2: 0.066299	valid_1's l2: 0.0643293
[2]	valid_0's l2: 0.0653766	valid_1's l2: 0.063435
[3]	valid_0's l2: 0.0644726	valid_1's l2: 0.0625585
[4]	valid_0's l2: 0.0635865	valid_1's l2: 0.0616995
[5]	valid_0's l2: 0.0627143	valid_1's l2: 0.0608552
[6]	valid_0's l2: 0.0618597	valid_1's l2: 0.060028
[7]	valid_

min_data_in_leaf, val_score: 0.019529: 100%|##########| 5/5 [00:19<00:00,  3.87s/it]


{'bagging_fraction': 0.6143236532771715,
 'bagging_freq': 4,
 'boosting': 'gbdt',
 'categorical_column': [22],
 'early_stopping_round': None,
 'feature_fraction': 0.9799999999999999,
 'feature_pre_filter': False,
 'lambda_l1': 4.0356377415232196e-05,
 'lambda_l2': 0.001441959093013018,
 'learning_rate': 0.01,
 'metric': 'l2',
 'min_child_samples': 25,
 'num_iterations': 1000,
 'num_leaves': 20,
 'num_threads': 1,
 'objective': 'regression',
 'seed': 42,
 'verbosity': -1}

In [ ]:
def prepare_datasets(df, fold):
    tr_df = df[df['fold']!=fold]
    val_df = df[df['fold']==fold]

    cat_cols_ = [f'{c}_encoded' for c in cat_cols]
    tr_ds = lgb.Dataset(tr_df[cols], tr_df['score'], categorical_feature=cat_cols_)
    val_ds = lgb.Dataset(val_df[cols], val_df['score'], categorical_feature=cat_cols_, reference=tr_ds)

    return tr_ds, val_ds


def train_fn(df, params):
    preds = []

    for fold in df['fold'].unique():
        tr_ds, val_ds = prepare_datasets(df, fold)
        
        model = lgb.train(
            params, 
            tr_ds, 
            valid_sets=[tr_ds, val_ds],
            callbacks=[lgb.early_stopping(stopping_rounds=100),
                       lgb.log_evaluation(period=100)]
        )

        pred = df[df['fold']==fold].copy()
        pred['pred_lgb'] = model.predict(pred[cols], num_iteration=model.best_iteration)
        preds.append(pred)

        model.save_model(output_dir / f'lgb_fold{fold}.txt', num_iteration=model.best_iteration)

    return pd.concat(preds, axis=0, ignore_index=False)

In [ ]:
pred = train_fn(train, best_params)

Training until validation scores don't improve for 100 rounds
[100]	training's l2: 0.0260689	valid_1's l2: 0.0250323
[200]	training's l2: 0.0203136	valid_1's l2: 0.0193365
[300]	training's l2: 0.0193159	valid_1's l2: 0.0185429
[400]	training's l2: 0.0190172	valid_1's l2: 0.0184608
[500]	training's l2: 0.0188438	valid_1's l2: 0.0184702
Early stopping, best iteration is:
[460]	training's l2: 0.0189101	valid_1's l2: 0.0184542
Training until validation scores don't improve for 100 rounds
[100]	training's l2: 0.0255954	valid_1's l2: 0.0264549
[200]	training's l2: 0.0197756	valid_1's l2: 0.0209589
[300]	training's l2: 0.0187898	valid_1's l2: 0.0201916
[400]	training's l2: 0.0184986	valid_1's l2: 0.0200975
[500]	training's l2: 0.0183326	valid_1's l2: 0.0201075
Early stopping, best iteration is:
[400]	training's l2: 0.0184986	valid_1's l2: 0.0200975
Training until validation scores don't improve for 100 rounds
[100]	training's l2: 0.0252412	valid_1's l2: 0.0269425
[200]	training's l2: 0.019345

In [ ]:
importances = []
for fold in train['fold'].unique():
    importances.append(
        lgb.Booster(model_file=output_dir / f'lgb_fold{fold}.txt').feature_importance(importance_type='gain'))
importance_df = pd.DataFrame({
    'feature': cols,
    'importance': np.mean(importances, axis=0)
    })
importance_df.sort_values('importance', ascending=False)

,feature,importance
10,pred_026,27445.891474
21,pred_022,13083.663228
22,section_encoded,149.782467
9,len_tgt_cnt_div_026,77.039187
7,len_anc_cnt_div_026,76.228503
3,input_len_026,69.661270
0,anchor_tok_len_026,61.804087
5,len_anc_tgt_div_026,60.066604
1,target_tok_len_026,43.562098
8,len_tgt_cnt_diff_026,43.403895


In [ ]:
# optunaのbest params
print(get_score(pred['score'], pred['pred_lgb']))
pred.head()

0.8402499900318942


,id,anchor,target,context,score,anchor_tok_len_026,target_tok_len_026,context_tok_len_026,input_len_026,len_anc_tgt_diff_026,len_anc_tgt_div_026,len_anc_cnt_diff_026,len_anc_cnt_div_026,len_tgt_cnt_diff_026,len_tgt_cnt_div_026,pred_026,anchor_tok_len_022,target_tok_len_022,context_tok_len_022,input_len_022,len_anc_tgt_diff_022,len_anc_tgt_div_022,len_anc_cnt_diff_022,len_anc_cnt_div_022,len_tgt_cnt_diff_022,len_tgt_cnt_div_022,pred_022,fold,section,section_encoded,pred_lgb
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,1,3,20,28,-2,0.333333,-19,0.05,-17,0.15,0.440476,1,3,20,28,-2,0.333333,-19,0.05,-17,0.15,0.126895,0,A,0,0.303510
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,1,5,20,30,-4,0.200000,-19,0.05,-15,0.25,0.641294,1,5,20,30,-4,0.200000,-19,0.05,-15,0.25,0.632196,0,A,0,0.627575
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0.247038,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0.237127,0,A,0,0.243537
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0.494176,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0.489984,0,A,0,0.480428
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0.011766,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0.013014,0,A,0,0.116151


In [ ]:
import os
import json
from kaggle.api.kaggle_api_extended import KaggleApi

ID = 'hanejiyuto'
DATASET_ID = ColabConfig.dataset_name + '-' + ColabConfig.dataset_version
UPLOAD_DIR = ColabConfig.dataset_dir
VERSION_NOTES = ColabConfig.dataset_note

def dataset_create_new():
    dataset_metadata = {}
    dataset_metadata['id'] = f'{ID}/{DATASET_ID}'
    dataset_metadata['licenses'] = [{'name': 'CC0-1.0'}]
    dataset_metadata['title'] = DATASET_ID
    with open(os.path.join(UPLOAD_DIR, 'dataset-metadata.json'), 'w') as f:
        json.dump(dataset_metadata, f, indent=4)
    api = KaggleApi()
    api.authenticate()
    # api.dataset_create_new(folder=UPLOAD_DIR, convert_to_csv=False, dir_mode='tar')
    !kaggle datasets create -t -p $UPLOAD_DIR -r tar

def dataset_create_version():  # バージョンアップデート
    dataset_metadata = {}
    dataset_metadata['id'] = f'{ID}/{DATASET_ID}'
    dataset_metadata['licenses'] = [{'name': 'CC0-1.0'}]
    dataset_metadata['title'] = DATASET_ID
    if not os.path.exists('dataset-metadata.json'):
        with open(os.path.join(UPLOAD_DIR, 'dataset-metadata.json'), 'w') as f:
            json.dump(dataset_metadata, f, indent=4)
    api = KaggleApi()
    api.authenticate()
    # api.dataset_create_version(folder=UPLOAD_DIR, version_notes=VERSION_NOTES, convert_to_csv=False, dir_mode='tar')
    !kaggle datasets version -t -p $UPLOAD_DIR -r tar -m $VERSION_NOTES

if CFG.debug:
    pass
elif ColabConfig.dataset_new:
    dataset_create_new()
else:
    dataset_create_version()